# ISIC API Encapsulation

In [1]:
import requests


class ISICApi(object):
    def __init__(self, hostname='https://isic-archive.com',
                 username=None, password=None):
        self.baseUrl = f'{hostname}/api/v1'
        self.authToken = None

        if username is not None:
            if password is None:
                password = input(f'Password for user "{username}":')
            self.authToken = self._login(username, password)

    def _makeUrl(self, endpoint):
        return f'{self.baseUrl}/{endpoint}'

    def _login(self, username, password):
        authResponse = requests.get(
            self._makeUrl('user/authentication'),
            auth=(username, password)
        )
        if not authResponse.ok:
            raise Exception(f'Login error: {authResponse.json()["message"]}')

        authToken = authResponse.json()['authToken']['token']
        return authToken

    def get(self, endpoint):
        url = self._makeUrl(endpoint)
        headers = {'Girder-Token': self.authToken} if self.authToken else None
        return requests.get(url, headers=headers)

    def getJson(self, endpoint):
        return self.get(endpoint).json()

    def getJsonList(self, endpoint):
        endpoint += '&' if '?' in endpoint else '?'
        LIMIT = 50
        offset = 0
        while True:
            resp = self.get(
                f'{endpoint}limit={LIMIT:d}&offset={offset:d}'
            ).json()
            if not resp:
                break
            for elem in resp:
                yield elem
            offset += LIMIT

## Connect to API

In [4]:
api = ISICApi(username="spaniernathan", password="4YwB3z&f78BjdBp4W2ghBt^k8YT&HZDS")

## Fetch images

In [6]:
import urllib
import os

savePath = 'ISICArchive/'

if not os.path.exists(savePath):
    os.makedirs(savePath)

imageList = api.getJson('image?limit=20&offset=0&sort=name')

print('Downloading %s images' % len(imageList))
imageDetails = []
for image in imageList:
    print(image['_id'])
    imageFileResp = api.get('image/%s/download' % image['_id'])
    imageFileResp.raise_for_status()
    imageFileOutputPath = os.path.join(savePath, '%s.jpg' % image['name'])
    with open(imageFileOutputPath, 'wb') as imageFileOutputStream:
        for chunk in imageFileResp:
            imageFileOutputStream.write(chunk)

5436e3abbae478396759f0cf
5436e3acbae478396759f0d1
5436e3acbae478396759f0d3
5436e3acbae478396759f0d5
5436e3acbae478396759f0d7
5436e3acbae478396759f0d9
5436e3acbae478396759f0db
5436e3acbae478396759f0dd
5436e3acbae478396759f0df
5436e3acbae478396759f0e1
5436e3acbae478396759f0e3
5436e3acbae478396759f0e5
5436e3acbae478396759f0e7
5436e3adbae478396759f0e9
5436e3adbae478396759f0eb
5436e3adbae478396759f0ed
5436e3adbae478396759f0ef
5436e3adbae478396759f0f1
5436e3adbae478396759f0f3
5436e3adbae478396759f0f5


## Fetch image details & metadata

In [23]:
print('Fetching metadata for %s images' % len(imageList))
imageDetails = []
for image in imageList:
    print(' ', image['name'])
    # Fetch the full image details
    imageDetail = api.getJson('image/%s' % image['_id'])
    imageDetails.append(imageDetail)
    print('  ', imageDetail['meta']['clinical'])
    print('')
    print('  ', imageDetail)
    print('')

Fetching metadata for 20 images
  ISIC_0000000
   {'age_approx': 55, 'anatom_site_general': 'anterior torso', 'benign_malignant': 'benign', 'diagnosis': 'nevus', 'diagnosis_confirm_type': None, 'melanocytic': True, 'sex': 'female'}

   {'_id': '5436e3abbae478396759f0cf', '_modelType': 'image', 'created': '2014-10-09T19:36:11.989000+00:00', 'creator': {'_id': '5450e996bae47865794e4d0d', 'name': 'User 6VSN'}, 'dataset': {'_accessLevel': -1, '_id': '5a2ecc5e1165975c945942a2', 'description': 'Moles and melanomas.\nBiopsy-confirmed melanocytic lesions. Both malignant and benign lesions are included.', 'license': 'CC-0', 'name': 'UDA-1', 'updated': '2014-11-10T02:39:56.492000+00:00'}, 'meta': {'acquisition': {'image_type': 'dermoscopic', 'pixelsX': 1022, 'pixelsY': 767}, 'clinical': {'age_approx': 55, 'anatom_site_general': 'anterior torso', 'benign_malignant': 'benign', 'diagnosis': 'nevus', 'diagnosis_confirm_type': None, 'melanocytic': True, 'sex': 'female'}}, 'name': 'ISIC_0000000', 'not

## Output metadata in CSV

In [27]:
import csv

outputFileName = 'imagedata'
outputFilePath = 'imagedata/'

if not os.path.exists(outputFilePath):
    os.makedirs(outputFilePath)

metadataFields = set(
    field
    for imageDetail in imageDetails
    for field in imageDetail['meta']['clinical']
)
metadataFields = ['isic_id'] + sorted(metadataFields)

print(metadataFields)

# Write the metadata to a CSV
print('Writing metadata to CSV: %s' % outputFileName+'.csv')

with open(outputFilePath+outputFileName+'.csv', 'w', newline='') as csvFile:
    csvWriter = csv.DictWriter(csvFile, fieldnames=metadataFields, extrasaction='ignore')
    csvWriter.writeheader()
    for imageDetail in imageDetails:
        rowDict = imageDetail['meta']['clinical']
        rowDict['isic_id'] = imageDetail['name']
        csvWriter.writerow(rowDict)

['isic_id', 'age_approx', 'anatom_site_general', 'benign_malignant', 'diagnosis', 'diagnosis_confirm_type', 'melanocytic', 'sex']
Writing metadata to CSV: imagedata.csv
